# IBM capstone project-The battle of Neighborhoods|Finding a Better medical resource and Higer live quality 

## installing and importing python libraries and dependencies

In [296]:
!pip install geocoder
!pip install folium
!pip install sklearn
!pip install geopy
!pip install bs4
!pip install lxml bs4 html5lib

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.


In [297]:
import pandas as pd
import requests
import numpy as np
import geocoder 
import folium
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings 
warnings.filterwarnings("ignore")

from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("All Required Libraries Imported!")

All Required Libraries Imported!


## Data Extraction and cleaning 

Using the list of Postal Codes of Given Wikipedia Page

In [298]:
Germany_url="https://raw.githubusercontent.com/zauberware/postal-codes-json-xml-csv/master/data/DE/zipcodes.de.json"
Germany=pd.read_json(Germany_url)
Switzerland_url="https://raw.githubusercontent.com/zauberware/postal-codes-json-xml-csv/master/data/CH/zipcodes.ch.json"
Switzerland=pd.read_json(Switzerland_url)

In [299]:
Germany.head()

,country_code,zipcode,place,state,state_code,province,province_code,community,community_code,latitude,longitude
0,DE,1945,Grünewald,Brandenburg,BB,,0,Landkreis Oberspreewald-Lausitz,12066,51.4000,14.0000
1,DE,1945,Lindenau,Brandenburg,BB,,0,Landkreis Oberspreewald-Lausitz,12066,51.4000,13.7333
2,DE,1945,Hohenbocka,Brandenburg,BB,,0,Landkreis Oberspreewald-Lausitz,12066,51.4310,14.0098
3,DE,1945,Schwarzbach,Brandenburg,BB,,0,Landkreis Oberspreewald-Lausitz,12066,51.4500,13.9333
4,DE,1945,Guteborn,Brandenburg,BB,,0,Landkreis Oberspreewald-Lausitz,12066,51.4167,13.9333


In [300]:
Germany.drop(['country_code','state_code','province','province_code','community_code','state'],axis = 'columns', inplace = True)

In [301]:
Germany.rename(columns={"zipcode":"postalcode","place":"neighborhoods","community":"Borough"},inplace=True)

In [302]:
Germany=Germany.dropna()
empty = 'Not assigned'
Germany=Germany[(Germany.postalcode!= empty ) & (Germany.Borough != empty) & (Germany.neighborhoods!= empty)]

In [303]:
Germany.head()

,postalcode,neighborhoods,Borough,latitude,longitude
0,1945,Grünewald,Landkreis Oberspreewald-Lausitz,51.4000,14.0000
1,1945,Lindenau,Landkreis Oberspreewald-Lausitz,51.4000,13.7333
2,1945,Hohenbocka,Landkreis Oberspreewald-Lausitz,51.4310,14.0098
3,1945,Schwarzbach,Landkreis Oberspreewald-Lausitz,51.4500,13.9333
4,1945,Guteborn,Landkreis Oberspreewald-Lausitz,51.4167,13.9333


In [304]:
Freiburg=Germany[Germany['Borough']=='Freiburg im Breisgau'].reset_index(drop=True)

In [305]:
Freiburg.head()

,postalcode,neighborhoods,Borough,latitude,longitude
0,79098,Freiburg im Breisgau,Freiburg im Breisgau,47.9942,7.8470
1,79100,Freiburg im Breisgau,Freiburg im Breisgau,47.9809,7.8327
2,79102,Freiburg im Breisgau,Freiburg im Breisgau,48.0000,7.8500
3,79104,Freiburg im Breisgau,Freiburg im Breisgau,48.0098,7.8545
4,79106,Freiburg im Breisgau,Freiburg im Breisgau,48.0000,7.8500


## Map of Freiburg

In [324]:
address = 'Freiburg im Breisgau,Freiburg im Breisgau'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude_x1 = location.latitude
longitude_y1 = location.longitude
print('The Geograpical Coordinate of Freibrug are {}, {}.'.format(latitude_x, longitude_y))

The Geograpical Coordinate of Freibrug are 47.3744489, 8.5410422.


In [325]:
map_Freiburg = folium.Map(location=[latitude_x, longitude_y], zoom_start=10)

for lat, lng, nei in zip(Freiburg['latitude'], Freiburg['longitude'], Freiburg['neighborhoods']):
    
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Freiburg)  
    
map_Freiburg

In [326]:
Switzerland.head()

,postalcode,neighborhoods,Borough,latitude,longitude
0,5000,Aarau,Aarau,47.3925,8.0442
1,5001,Aarau 1,Aarau,47.3888,8.0483
2,5004,Aarau,Aarau,47.3925,8.0442
3,5017,Barmelweid,Erlinsbach (AG),47.4159,7.9764
4,5018,Erlinsbach,Erlinsbach (AG),47.4052,8.0151


In [327]:
Switzerland.drop(['country_code','state_code','province','province_code','community_code','state'],axis = 'columns', inplace = True)

KeyError: "['country_code' 'state_code' 'province' 'province_code' 'community_code'\n 'state'] not found in axis"

In [328]:
Switzerland.rename(columns={"zipcode":"postalcode","place":"neighborhoods","community":"Borough"},inplace=True)

In [329]:
Switzerland=Switzerland.dropna()
empty = 'Not assigned'
Switzerland=Switzerland[(Switzerland.postalcode!= empty ) & (Switzerland.Borough != empty) & (Switzerland.neighborhoods!= empty)]

In [330]:
Switzerland.head()

,postalcode,neighborhoods,Borough,latitude,longitude
0,5000,Aarau,Aarau,47.3925,8.0442
1,5001,Aarau 1,Aarau,47.3888,8.0483
2,5004,Aarau,Aarau,47.3925,8.0442
3,5017,Barmelweid,Erlinsbach (AG),47.4159,7.9764
4,5018,Erlinsbach,Erlinsbach (AG),47.4052,8.0151


In [331]:
Zürich=Switzerland[Switzerland['Borough']=='Zürich'].reset_index(drop=True)

In [332]:
Zürich.head()

,postalcode,neighborhoods,Borough,latitude,longitude
0,8000,Zürich,Zürich,47.3828,8.5307
1,8001,Zürich,Zürich,47.3667,8.5500
2,8002,Zürich,Zürich,47.3667,8.5500
3,8003,Zürich,Zürich,47.3667,8.5500
4,8004,Zürich,Zürich,47.3667,8.5500


## Map of Zürich 

In [333]:
address = 'Zürich,Zürich'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude_x2 = location.latitude
longitude_y2 = location.longitude
print('The Geograpical Coordinate of Zürich are {}, {}.'.format(latitude_x, longitude_y))

The Geograpical Coordinate of Zürich are 47.3744489, 8.5410422.


In [334]:
map_Zurich = folium.Map(location=[latitude_x, longitude_y], zoom_start=10)

for lat, lng, nei in zip(Zürich['latitude'], Zürich['longitude'], Zürich['neighborhoods']):
    
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Zurich)  
    
map_Zurich

In [378]:
CLIENT_ID='4DUPT32JSBF4RGHB40QAJJ3SKQG5GA1X52RPJTGUKXU23EHO'
CLIENT_SECRET='WB5CZBEHI43JJLOUJXBXJN53AL11QTLLLP1SRP02GDHG4IQK'
VERSION='20210215'
LIMIT=30
print('Your credentials:')
print('CLIENT_ID:'+CLIENT_ID)
print('CLIENT_SECRET:'+CLIENT_SECRET)

Your credentials:
CLIENT_ID:4DUPT32JSBF4RGHB40QAJJ3SKQG5GA1X52RPJTGUKXU23EHO
CLIENT_SECRET:WB5CZBEHI43JJLOUJXBXJN53AL11QTLLLP1SRP02GDHG4IQK


In [387]:
radius = 700 
LIMIT = 100
url1= 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_x1, 
   longitude_y1, 
    radius, 
   LIMIT)
Freiburg_results = requests.get(url1).json()

In [388]:
Freiburg_venues=Freiburg_results['response']['groups'][0]['items']
Freiburg_nearby_venues = json_normalize(venues)
Freiburg_nearby_venues.columns

Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.lat',
       'venue.location.lng', 'venue.location.labeledLatLngs',
       'venue.location.distance', 'venue.location.postalCode',
       'venue.location.cc', 'venue.location.city', 'venue.location.state',
       'venue.location.country', 'venue.location.formattedAddress',
       'venue.categories', 'venue.photos.count', 'venue.photos.groups',
       'venue.location.crossStreet', 'venue.venuePage.id',
       'venue.location.neighborhood'],
      dtype='object')

In [389]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Freiburg Nearby Venues/Locations

In [390]:
Freiburg_filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Freiburg_nearby_venues =nearby_venues.loc[:, filtered_columns]
Freiburg_nearby_venues.head()


,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Schloß-Café,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",47.981173,7.838582
1,Lollo,"[{'id': '4bf58dd8d48988d10d941735', 'name': 'G...",47.984859,7.841542
2,Lorettoberg,"[{'id': '4eb1d4d54b900d56c88a45fc', 'name': 'M...",47.976234,7.837640
3,H Peter-Thumb-Straße,"[{'id': '52f2ab2ebcbc57f1066b8b51', 'name': 'T...",47.979782,7.831986
4,Agip Service Station,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",47.979668,7.831522


In [409]:
radius = 700 
LIMIT = 100
url2= 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_x2, 
   longitude_y2, 
    radius, 
   LIMIT)
Zürich_results = requests.get(url2).json()

In [410]:
Zürich_venues=Zürich_results['response']['groups'][0]['items']
Zürich_nearby_venues = json_normalize(venues)
Zürich_nearby_venues.columns

Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.lat',
       'venue.location.lng', 'venue.location.labeledLatLngs',
       'venue.location.distance', 'venue.location.postalCode',
       'venue.location.cc', 'venue.location.city', 'venue.location.state',
       'venue.location.country', 'venue.location.formattedAddress',
       'venue.categories', 'venue.photos.count', 'venue.photos.groups',
       'venue.location.crossStreet', 'venue.venuePage.id',
       'venue.location.neighborhood'],
      dtype='object')

In [411]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [412]:
Zürich_filtered_columns = ['z_venue.name', 'z_venue.categories', 'z_venue.location.lat', 'z_venue.location.lng']
Zürich_nearby_venues =nearby_venues.loc[:, filtered_columns]
Zürich_nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Schloß-Café,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",47.981173,7.838582
1,Lollo,"[{'id': '4bf58dd8d48988d10d941735', 'name': 'G...",47.984859,7.841542
2,Lorettoberg,"[{'id': '4eb1d4d54b900d56c88a45fc', 'name': 'M...",47.976234,7.837640
3,H Peter-Thumb-Straße,"[{'id': '52f2ab2ebcbc57f1066b8b51', 'name': 'T...",47.979782,7.831986
4,Agip Service Station,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",47.979668,7.831522
